In [5]:
"""A very simple MNIST classifier.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/beginners
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None

def swish(x):
  return x*tf.nn.sigmoid(x)

def selu(x):
  with tf.name_scope('elu') as scope:
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    return scale*tf.where(x>=0.0, x, alpha*tf.nn.elu(x))

def activation(x):
#  return selu(x)
#  return relu(x)
#  return sigmoid(x)
  return swish(x)

def initialize(shape, stddev=0.1):
  return tf.truncated_normal(shape, stddev=0.1)


# Import data
data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

learning_rate = tf.placeholder(tf.float32)
  
# Create the model
L1_units_count = 100
x = tf.placeholder(tf.float32, [None, 784])
W_1 = tf.Variable(initialize([784, L1_units_count], stddev=0.05))
b_1 = tf.Variable(initialize([L1_units_count]))
logits_1 = tf.matmul(x, W_1) + b_1
output_1 = activation(logits_1)

L2_units_count = 10 
W_2 = tf.Variable(initialize([L1_units_count, L2_units_count], stddev=0.063))
b_2 = tf.Variable(initialize([L2_units_count]))
logits_2 = tf.matmul(output_1, W_2) + b_2  

y = logits_2



# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
l2_loss = tf.nn.l2_loss(W_1) + tf.nn.l2_loss(W_2)
total_loss = cross_entropy + 7e-5*l2_loss
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

# 还记得不？不要把graph定义写到图里面
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Train
for step in range(3000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  if step<1800:
      lr = 1.2
  elif step<3400:
      lr = 0.3
  else:
      lr = 0.01
  _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr})
  
  if (step+1) % 100 == 0:
    print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
    print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys}))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels}))

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz
step 100, entropy loss: 0.436885, l2_loss: 424.297852, total loss: 0.466586
0.93
0.898
step 200, entropy loss: 0.227865, l2_loss: 441.455200, total loss: 0.258767
0.98
0.9236
step 300, entropy loss: 0.185471, l2_loss: 451.854950, total loss: 0.217101
0.99
0.9259
step 400, entropy loss: 0.147222, l2_loss: 459.830048, total loss: 0.179410
0.98
0.9469
step 500, entropy loss: 0.103422, l2_loss: 466.429260, total loss: 0.136073
1.0
0.9472
step 600, entropy loss: 0.121131, l2_loss: 471.787720, total loss: 0.154156
0.97
0.9562
step 700, entropy loss: 0.077988, l2_loss: 475.861115, total loss: 0.111298
1.0
0.9536
step 800, entropy loss: 0.088596, l2_loss: 480.307556, total loss: 0.122218
0.99
0.9561
step 900, entropy